In [1]:
#Movie Recommendation Data Warehouse Project

In [2]:
# #Connection to SQL DataBase
# import mysql.connector

# # Define the database connection details
# host = 'localhost'
# port = '3306'
# database = 'sys'
# user = 'root'
# password = 'my_password'

# # Connect to the database
# conn = mysql.connector.connect(host=host, port=port, database=database, user=user, password=password)

In [3]:
# cur = conn.cursor()

# cur.execute("""
# CREATE TABLE users (
#   user_id INT PRIMARY KEY,
#   name VARCHAR(255),
#   email VARCHAR(255),
#   created_at DATETIME
# );

# CREATE TABLE watch_history (
#     user_id INT,
#     movie_id INT,
#     rating INT,
#     watch_date DATE
# );

# CREATE TABLE movie_data (
#     index INT,
#     budget INT,
#     genres TEXT,
#     id INT,
#     keywords TEXT,
#     original_title TEXT,
#     overview TEXT,
#     popularity FLOAT,
#     release_date DATE,
#     revenue INT,
#     runtime FLOAT,
#     spoken_languages TEXT,
#     tagline TEXT,
#     title TEXT,
#     vote_average FLOAT,
#     vote_count INT,
#     cast TEXT,
#     director TEXT,
#     features TEXT
# );


# CREATE VIEW user_movie_ratings AS
# SELECT u.username, m.title, w.rating
# FROM user_data u
# INNER JOIN watch_history w ON u.id = w.user_id
# INNER JOIN movies_data m ON w.movie_id = m.id;



# """)


# # Commit the changes to the database
# conn.commit()

# # Close the cursor and the database connection
# cur.close()
# conn.close()

In [4]:
# cur.execute = ("""
# INSERT INTO user_data (username, password, email, age, gender)
# VALUES ('Vijay', 'mypassword123', 'Vijay@example.com', 51, 'Male'),
#        ('vedanshi', 'mypassword456', 'vedanshi@example.com', 23, 'Female'),
#        ('Jyoti', 'password', 'jai@example.com', 49, 'Female'),
#        ('Shivank', 'mypassword789', 'shivank@example.com', 21, 'Male');
       
# INSERT INTO watch_history (user_id, movie_id, rating)
# VALUES (1, 2, 4),
#        (1, 3, 3),
#        (1, 1, 5),
#        (2, 4, 2),
#        (4, 1, 3),
#        (4, 3, 4);

       
       
# """)
# # Commit the changes to the database
# conn.commit()

# # Close the cursor and the database connection
# cur.close()
# conn.close()

       


    

In [6]:
# Fetching the movies which user 1 rated

In [7]:
# cur.execute = ("""

# Select * from dbo.[user_movie_ratings] where user_id = 1;
# watched_movies = cur.fetchall()

# """)

# # Close the cursor and the database connection
# cur.close()
# conn.close()


In [8]:
#importing necessary Libraries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
#importing Data
df = pd.read_csv('movies.csv')

In [10]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [11]:
#Filling Null Values 
df = df.fillna(' ')

In [12]:
#Making New Column For Main Features
df['Features'] = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [13]:
#Dropping unnecessary Columns
df.drop(['homepage', 'crew', 'status', 'original_language', 'production_companies', 'production_countries'], axis=1, inplace=True)

In [14]:
# movies user has already watched
watched_movies = ['Iron Man', 'Bat Man', 'Avengers']

In [15]:
#Removing those Movies from Our data Frame
df = df[~df['title'].isin(watched_movies)]

In [16]:
#Making those Features into Vector
features = df['Features']
vec = TfidfVectorizer()
feature_vector = vec.fit_transform(features)

In [17]:
#Finding Cosine Similarity
cs = cosine_similarity(feature_vector)


In [19]:
def suggest_movies(movie_name):
    # Find the closest matching movie name in the list
    find_close_match = difflib.get_close_matches(movie_name, df['title'].tolist())
    close_match = find_close_match[0]

    # Find the index of the closest matching movie in the DataFrame
    index_movie = df[df.title == close_match].index[0]

    # Calculate the similarity score between the closest matching movie and all other movies in the DataFrame
    similarity_score = list(enumerate(cs[index_movie]))

    # Sort the movies by similarity score
    sorted_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # Print the top 30 suggested movies
    suggested_movies = []
    i = 1
    for movie in sorted_movies:
        index = movie[0]
        title_from_index = df.iloc[index]['title']
        if (i < 6):
            suggested_movies.append(title_from_index)
            i += 1
        else:
            break
    
    return suggested_movies

In [20]:
# Get movie recommendations for the viewer
watched_movies_recommendations = []
for watched_movie in watched_movies:
    watched_movie_recommendations = suggest_movies(watched_movie)
    watched_movies_recommendations.extend(watched_movie_recommendations)

In [24]:
print("Here are some recommended movies for you based on your watch history")
for i, movie in enumerate(set(watched_movies_recommendations)):
    print(f"{i+1}. {movie}")

Here are some recommended movies for you based on your watch history
1. Knight and Day
2. Criminal
3. Bad Boys II
4. The Avengers
5. Captain America: The Winter Soldier
6. Captain America: Civil War
7. Riddick
8. There Be Dragons
9. Avengers: Age of Ultron
10. Iron Man 2
11. Iron Man 3
